### This is a framework to test if the API docs are retrievable for the given API

In [6]:
from utilities import fetch_api_docs
import time
def test_api_discovery():
    test_apis = {
        # OpenWeather
        "Weather": {"url" : "https://openweathermap.org/api",
        "API_KEY" : "3c13f2d6e8beb2b1f742324dbbd8b212"},
        # 2. Reddit
        "Reddit": {"url" : "https://www.reddit.com/dev/api/",
        "API_KEY" : None    },
        # 3. OpenExchangeRates  
        "Currency": {"url" : "https://docs.openexchangerates.org/reference/api-introduction",
        "API_KEY" : None},
        # 4. NASA API 
        "NASA": {"url" : "https://api.nasa.gov/",
        "API_KEY" : "ISxBTHQjvQaazVnIHy6322mahQx6fIm9YHzcFuMB"},
        # 5. News-API 
        "News": {"url" : "https://newsapi.org/docs/client-libraries/python",
        "API_KEY" : "17a8051fc49f4cae89ca006f5e4189c6"},
    }
    
    for api, info in test_apis.items():
        print(f"\nTesting {api}")
        result = fetch_api_docs(info["url"])
        if result:
            print("="*100)
            print(f" Yes Documentation found")
            # To print first 100 characters of result uncommnt this
            print(f"Preview: {result[:100]}...")
        else:
            print("="*100)
            print("No documentation found")
            print("="*100)
        time.sleep(1)  

test_api_discovery()


Testing Weather
 Yes Documentation found
Preview: <!DOCTYPE html>
<html lang="en">

<head>
  <title> Weather API - OpenWeatherMap</title>
  <meta name...

Testing Reddit
 Yes Documentation found
Preview: <!doctype html><html xmlns="http://www.w3.org/1999/xhtml" lang="en" xml:lang="en"><head><title>reddi...

Testing Currency
 Yes Documentation found
Preview: <!DOCTYPE html><html lang="en" style="" data-color-mode="light" class=" useReactApp isRefPage "><hea...

Testing NASA
 Yes Documentation found
Preview: <!doctype html>
<base href="#">
<html lang="en">
<head>
  <meta charset="utf-8">
  <title>NASA Open ...

Testing News
 Yes Documentation found
Preview: 

<!DOCTYPE html>

<html lang="en">
<head>
  <title>Python client library - News API</title>
...


### Check if you are able to get response from Claude through AWS Bedrock

In [24]:
import requests
try:
    response = requests.post(
        "http://localhost:8000/chat_claude",
        json={"text": "Say hi in one word"}
    )
    response.raise_for_status()
    print(response.json()["response"])
except requests.exceptions.RequestException as e:
    print(f"Error: {e}")

Hi.


### Check if you are able to get response from Titan through AWS Bedrock

In [25]:
import requests
try:
    response = requests.post(
        "http://localhost:8000/chat_titan",
        json={"text": "Say hi in one word"}
    )
    response.raise_for_status()
    print(response.json()["response"])
except requests.exceptions.RequestException as e:
    print(f"Error: {e}")


Hi!


### Check if the API docs are fetched correctly

In [2]:
import requests
from utilities import test_apis
base_url = "http://localhost:8000"

# Fetch the API docs for the given API name from utilities.py
api_name = "Currency"
params={"api_docs_url": test_apis[api_name]['url']}

weather_response = requests.get(f"{base_url}/weather", params=params)
print(weather_response.json()["response"])

<!DOCTYPE html><html lang="en" style="" data-color-mode="light" class=" useReactApp isRefPage "><head><script src="https://cdn.readme.io/public/js/force-firefox-anchor-jump.js?1739992711721"></script><meta charset="utf-8"><meta http-equiv="X-UA-Compatible" content="IE=edge"><meta name="readme-deploy" content="5.284.0"><meta name="readme-subdomain" content="oxr"><meta name="readme-version" content="0.7"><title>API Introduction</title><meta name="description" content="Open Exchange Rates provides a simple, lightweight and portable JSON API with live and historical foreign exchange (forex) rates, via a simple and easy-to-integrate API, in JSON format. Data are tracked and blended algorithmically from multiple reliable sources, ensuring fair and unbiased consistenc..."><meta property="og:title" content="API Introduction"><meta property="og:description" content="Open Exchange Rates provides a simple, lightweight and portable JSON API with live and historical foreign exchange (forex) rates, 

### Check if the user query is fetched correctly

In [27]:
# Method 1: Using params
base_url = "http://localhost:8000"
params = {"user_query": "What is the weather in London?"}
response = requests.post(f"{base_url}/user_query", params=params)
print(response.json())

{'response': 'What is the weather in London?'}


### Test the query with Claude as the mother agent and Titan as the child agent

In [6]:
from utilities import test_apis

base_url = "http://localhost:8000"

# Fetch the API docs for the given API name from utilities.py
api_name = "Weather"
params = {
    "user_query": "What is the weather like at latitude 37.7749 and longitude -122.4194",
    "api_name": api_name
}

query_response = requests.post(f"{base_url}/user_query", params = params)
print(query_response.json())

response = requests.get(f"{base_url}/titan_child_agent", params=params)
print("Agent Response:", response.json())

{'response': 'What is the weather like at latitude 37.7749 and longitude -122.4194'}
Agent Response: {'message': 'Child agent successfully created', 'response': 'import boto3\nimport requests\nimport json\nimport os\nfrom bedrock_handler import call_titan\nfrom dotenv import load_dotenv\n\nload_dotenv()\n\napi_key = os.getenv("OPENWEATHER_API_KEY")\n\ndef get_data():\n    lat = 37.7749\n    lon = -122.4194\n    \n    base_url = "https://api.openweathermap.org/data/2.5/weather"\n    \n    params = {\n        "lat": lat,\n        "lon": lon,\n        "appid": api_key,\n        "units": "metric"\n    }\n    \n    try:\n        response = requests.get(base_url, params=params)\n        response.raise_for_status()\n        data = response.json()\n        \n        temp = data["main"]["temp"]\n        description = data["weather"][0]["description"]\n        \n        weather_data = f"Temperature: {temp} C, Weather: {description}"\n        return weather_data\n    \n    except requests.excepti

### Added a route to test and verify the health of the child agent code and execute it to get the response from the child agent

The route verifies to see if the child agent code:
- Is syntactically valid
- Can be executed
- Returns a valid response

In [7]:
import requests

# The path to the agent file you want to verify
agent_file = "child_agent.py"

# Make the POST request
response = requests.post(f"http://localhost:8000/verify_agent?agent_file={agent_file}")

# Print the response
print(response.json())

{'status': 'success', 'syntax_valid': True, 'message': 'Agent file executed successfully', 'output': '{URL-1}\n```\nIn Python, to resolve the 401 Client Error: Unauthorized for url: {URL-2} error, you can use the below code:\n```Python\nimport requests\nimport json\ntry:\n    # Define the API endpoint URL\n    url = "https://api.openweathermap.org/data/2.5/weather?lat=37.7749&lon=-122\n'}


In [ ]:
import child_agent
!python child_agent.py